In [8]:
import math

### Arduino
[Uno R3 schematic](http://www.arduino.cc/de/uploads/Main/Arduino_Uno_Rev3-schematic.pdf)

[atmega datasheet](http://www.atmel.com/images/Atmel-8271-8-bit-AVR-Microcontroller-ATmega48A-48PA-88A-88PA-168A-168PA-328-328P_datasheet_Complete.pdf)

### Zero-crossing detector

AC optocoupler PC824
[datasheet](https://www.pollin.de/shop/downloads/D121107D.PDF)

http://playground.arduino.cc/Main/ACPhaseControl

Choose input current of 5mA according to Fig. 5 in datasheet:

In [3]:
# LED current used in AC phase control at 110V 
# I = U / R
math.sqrt(2) * 110 / (2 * 15e3)

0.005185449728701348

In [4]:
# using the same LED current
i_f = 5e-3

230V input voltage. 

Background information on calculating input resistances:
https://electronics.stackexchange.com/questions/33042/how-do-i-select-the-accompanying-components-for-an-optocoupler

In [5]:
u_ac = 230
u_peak = math.sqrt(2) * u_ac
u_peak

325.2691193458119

Input resistor size

In [6]:
r = u_peak / i_f
r

65053.82386916237

In [7]:
#Using 2 resistors of 33k
r = 12e3

In [8]:
i_f = u_peak / (2 * r)
i_f

0.013552879972742161

In [9]:
# power consumption
# P = U * I
# P = U^2 / R
p = u_ac**2 / (2*r)
p

2.2041666666666666

so 2 resistors of 33k, 0.5W should be fine

### solid state relay

[datasheet](http://www.pollin.de/shop/downloads/D340470D.PDF)

[china datasheet](http://www.chinaxuruien.com/product-129.html)
built-in zero crossing detector

In [1]:
325./12e3

0.027083333333333334

### interrupt timing

In [3]:
avr_frequency = 16e6
uint16_max = 2**16-1
ac_frequency = 50

In [4]:
cycle_time = 1./avr_frequency
cycle_time

6.25e-08

In [5]:
ac_halfwave_time = (1./(2*ac_frequency))
ac_halfwave_time

0.01

In [6]:
cycles_per_half_wave = ac_halfwave_time / cycle_time
cycles_per_half_wave

160000.0

In [10]:
divisor = cycles_per_half_wave / uint16_max
min_pow2_divisor = 2**int(math.ceil(math.log(divisor,2)))
min_pow2_divisor

4

In [21]:
2 * cycles_per_half_wave / 256

1250.0

In [22]:
0.01 / (cycle_time * 256)

625.0

In [24]:
2**16 / 1250

52

### keypad shield

[datasheet](http://www.arduino-projekte.de/index.php?n=25)

[datasheet](http://www.exp-tech.de/media/archive/gm/service/datasheet/LCD1602shield.pdf)

[datasheet](http://www.discoveringelectronics.com/Downloads/Sain-Smart-LCD-Keypad-Shield-v1.0-Pinout.pdf)

### ds18b20

#### get addresses

/* YourDuino Example: Find Address of a DS18B20 Temperature Sensor
 Cut and paste the address to a text file for later use.
 V1.1 01/17/2013
 Questions: terry@yourduino.com 
 
 Connections:
 DS18B20 Pinout (Left to Right, pins down, flat side toward you)
 - Left   = Ground
 - Center = Signal (Pin 2):  (with 3.3K to 4.7K resistor to +5 or 3.3 )
 - Right  = +5 or +3.3 V   
 This sketch looks for 1-wire devices and  prints their addresses (serial number)
 to the Serial Monitor in a format that is useful in Arduino sketches.
 Based on example at: 
 http://www.hacktronics.com/Tutorials/arduino-1-wire-address-finder.html
 */

/*-----( Import needed libraries )-----*/
#include <OneWire.h>

/*-----( Declare Constants and Pin Numbers )-----*/
#define SENSOR_PIN 11  // Any pin 2 to 12 (not 13) and A0 to A5

/*-----( Declare objects )-----*/
OneWire  ourBus(SENSOR_PIN);  // Create a 1-wire object

void setup()  /****** SETUP: RUNS ONCE ******/
{
  Serial.begin(9600);
 
  discoverOneWireDevices();  // Everything happens here!
}//--(end setup )---

void loop()   /****** LOOP: RUNS CONSTANTLY ******/
{
  // Nothing happening here
}

/*-----( Declare User-written Functions )-----*/
void discoverOneWireDevices(void) {
  byte i;
  byte present = 0;
  byte data[12];
  byte addr[8];

  Serial.print("Looking for 1-Wire devices...\n\r");// "\n\r" is NewLine 
  while(ourBus.search(addr)) {
    Serial.print("\n\r\n\rFound \'1-Wire\' device with address:\n\r");
    for( i = 0; i < 8; i++) {
      Serial.print("0x");
      if (addr[i] < 16) {
        Serial.print('0');
      }
      Serial.print(addr[i], HEX);
      if (i < 7) {
        Serial.print(", ");
      }
    }
    if ( OneWire::crc8( addr, 7) != addr[7]) {
      Serial.print("CRC is not valid!\n\r");
      return;
    }
  }
  Serial.println();
  Serial.print("Done");
  ourBus.reset_search();
  return;
}

My sensors

```0: 0x28, 0xFF, 0x24, 0x79, 0x01, 0x15, 0x02, 0x90
1: 0x28, 0xFF, 0x95, 0x94, 0x01, 0x15, 0x02, 0xC6
2: 0x28, 0xFF, 0xE7, 0x57, 0x01, 0x15, 0x02, 0x99```

Temperature sensors connected to pin 21
currently connected sensor 0


sensor 0: water
opt sensor 1: meat

```
/dev/serial/by-id$ stty -F usb-Arduino__www.arduino.cc__0043_75439333635351A02202-if00 -a
speed 115200 baud; rows 0; columns 0; line = 0;
intr = ^C; quit = ^\; erase = ^?; kill = ^U; eof = ^D; eol = <undef>; eol2 = <undef>;
swtch = <undef>; start = ^Q; stop = ^S; susp = ^Z; rprnt = ^R; werase = ^W; lnext = ^V;
discard = ^O; min = 1; time = 0;
-parenb -parodd -cmspar cs8 hupcl -cstopb cread clocal -crtscts
-ignbrk -brkint -ignpar -parmrk -inpck -istrip -inlcr -igncr icrnl ixon -ixoff -iuclc -ixany
-imaxbel -iutf8
opost -olcuc -ocrnl -onlcr -onocr -onlret -ofill -ofdel nl0 cr0 tab0 bs0 vt0 ff0
isig icanon iexten -echo echoe echok -echonl -noflsh -xcase -tostop -echoprt echoctl echoke
-flusho -extproc
```